In [67]:
import time
from pathlib import Path

import vk
import os
from pyvis.network import Network
from dotenv import load_dotenv

In [68]:
load_dotenv(Path('config/.env'))

token = ''
try:
    token = os.environ['VK_ACCESS_TOKEN']
except KeyError:
    print("You need to enter your access token from https://vkhost.github.io/ into env VK_ACCESS_TOKEN")
    exit(1)

vk_api = vk.API(token, v='5.131')

In [69]:
max_friends_per_friend = 100
def get_user_graph(user_id):
    friends = vk_api.friends.get(user_id=user_id)['items']
    friends_of_friends_dict = {}
    friends_set = set()
    friends_edges_set = set()

    for friend in friends:
        try:
            friends_of_friends = vk_api.friends.get(user_id=friend)['items']
        except Exception:
            continue # если пользователь удален
        if len(friends_of_friends) > max_friends_per_friend:
            continue
        friends_of_friends_dict[friend] = friends_of_friends
        
        friends_set.add(friend)
        for friend_of_friend in friends_of_friends_dict[friend]:
            friends_set.add(friend_of_friend)
            friend_pair = [friend, friend_of_friend]
            friend_pair.sort()
            friends_edges_set.add(tuple(friend_pair))
        time.sleep(0.2) # потому что ограничение на количество запросов в секунду

    return friends_set, friends_edges_set

In [70]:
id_anton = 243317609
id_matvey = 207227130

anton_friend_nodes, anton_friend_edges = get_user_graph(id_anton)
matvey_friend_nodes, matvey_friend_edges = get_user_graph(id_matvey)

In [71]:
friend_set = anton_friend_nodes.union(matvey_friend_nodes)
friend_edge_set = anton_friend_edges.union(matvey_friend_edges)

In [72]:
networkx_graph = nx.Graph()
networkx_graph.add_nodes_from(friend_set)
networkx_graph.add_edges_from(friend_edge_set)

In [73]:
degree_centrality = list(nx.degree_centrality(networkx_graph).items())
closeness_centrality = list(nx.closeness_centrality(networkx_graph).items())
betweenness_centrality = list(nx.betweenness_centrality(networkx_graph).items())
eigenvector_centrality = list(nx.eigenvector_centrality(networkx_graph, max_iter=600).items())
pagerank = list(nx.pagerank(networkx_graph, alpha=0.85, max_iter=600).items())

def make_dict(nodes_with_metric):
    result = {}
    for index, element in nodes_with_metric:
        result[index] = element
    return result

degree_centrality_dict = make_dict(degree_centrality)
closeness_centrality_dict = make_dict(closeness_centrality)
betweenness_centrality_dict = make_dict(betweenness_centrality)
eigenvector_centrality_dict = make_dict(eigenvector_centrality)
pagerank_dict = make_dict(pagerank)

In [74]:
def make_pyvis_graph(metric_dict, min_metric, max_metric):
    pyvis_graph = Network(notebook=True, height=2000)

    for friend in friend_set:
        min_size = 10
        size_factor = 30
        size = min_size + (size_factor * (metric_dict[friend] - min_metric) / (max_metric - min_metric))

        if friend == id_anton:
            color = 'black'
        elif friend == id_matvey:
            color = 'cyan'
        elif friend in anton_friend_nodes and friend in matvey_friend_nodes:
            color = 'purple'
        elif friend in anton_friend_nodes:
            color = 'red'
        elif friend in matvey_friend_nodes:
            color = 'blue'

        pyvis_graph.add_node(friend, label=str(friend) + '\n' + str(metric_dict[friend]), color = color, size = size)

    for friend_pair in friend_edge_set:
        pyvis_graph.add_edge(friend_pair[0], friend_pair[1])

    return pyvis_graph

In [75]:
pyvis_degree_centrality_graph = make_pyvis_graph(degree_centrality_dict, min(degree_centrality_dict.values()), max(degree_centrality_dict.values()))
pyvis_closeness_centrality_graph = make_pyvis_graph(closeness_centrality_dict, min(closeness_centrality_dict.values()), max(closeness_centrality_dict.values()))
pyvis_betweenness_centrality_graph = make_pyvis_graph(betweenness_centrality_dict, min(betweenness_centrality_dict.values()), max(betweenness_centrality_dict.values()))
pyvis_eigenvector_centrality_graph = make_pyvis_graph(eigenvector_centrality_dict, min(eigenvector_centrality_dict.values()), max(eigenvector_centrality_dict.values()))
pyvis_pagerank_graph = make_pyvis_graph(pagerank_dict, min(pagerank_dict.values()), max(pagerank_dict.values()))

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [84]:
pyvis_degree_centrality_graph.show(str(Path('degree_centrality.html')))
pyvis_closeness_centrality_graph.show(str(Path('closeness_centrality.html')))
pyvis_betweenness_centrality_graph.show(str(Path('betweenness_centrality.html')))
pyvis_eigenvector_centrality_graph.show(str(Path('eigenvector_centrality.html')))
pyvis_pagerank_graph.show(str(Path('pagerank.html')))